# Part A

A lot of the graph/vertex class was taken directly from PS5 solution.

**See README for part A description.**

In [1]:
from math import inf

In [2]:
class Vertex:
    
    def __init__(self,id):
        self.attributes = {}
        self.attributes['id'] = id
        
    def __str__(self):
        return str(self.attributes)
        
    def new_copy(self):
        return Vertex(self.attributes['id'])
        
    def set(self,key,value):
        self.attributes[key] = value
        
    def get(self,key):
        return self.attributes[key]

In [3]:
class Graph:
    
    def __init__(self):
        self.vertices = {}
        self.id_to_v = {}
        self.edge_attributes = {}
    
    def __str__(self):
        s = ''
        for v in self.vertices:
            s += str(v)
            s += '\n\n'
        return s
    
    def add_vertex(self,v):
        self.vertices[v] = []
        self.id_to_v[v.get('id')] = v
    
    def size_vertices(self):
        return len(self.vertices)
    
    def add_edge(self,v1,v2):
        pass
    
    def adjacent(self,v):
        return self.vertices[v]
    
    def BFS(self,start):
        assert(isinstance(start,Vertex))
        result = []
        
        start.set('color','gray')
        start.set('d',0)
        start.set('parent',None)
        
        for v in self.vertices:
            if v == start:
                continue
            
            v.set('color','white')
            v.set('d',inf)
            v.set('parent',None)
        
        queue = []
        queue.append(start)
        
        while len(queue) != 0:
            
            u = queue.pop(0)
            
            for v in self.adjacent(u):
                
                if v.get('color') == 'white':
                    
                    v.set('color','gray')
                    v.set('d',u.get('d') + 1)
                    v.set('parent',u)
                    
                    queue.append(v)
                    
            u.set('color','black')
            result.append(u.get('id'))

        return result
    
    def DFS_Visit(self,u,time):
        
        time += 1
        u.set('d',time)
        u.set('color','gray')
    
        for v in self.adjacent(u):
            
            if v.get('color') == 'white':
                v.set('parent',u)
                time = self.DFS_Visit(v,time)
    
        u.set('color','black')
        time += 1
        u.set('f',time)
    
        return time
    
    def DFS(self):
        
        for v in self.vertices:
            
            v.set('color','white')
            v.set('parent',None)
            
        time = 0
        
        for v in self.vertices:
            
            if v.get('color') == 'white':
                time = self.DFS_Visit(v,time)         

In [4]:
class UndirectedGraph(Graph):
        
    def add_edge(self,v1,v2):
        self.vertices[v1].append(v2)
        self.vertices[v2].append(v1)

In [5]:
class DirectedGraph(Graph):
        
    def add_edge(self,v1,v2, s):
        self.vertices[v1].append(v2)
        self.edge_attributes[(v1,v2)] = {}
        self.edge_attributes[(v1,v2)]['sound'] = s 
        
    def transpose(self):
        GT = DirectedGraph()
        
        for v in self.vertices:
            GT.add_vertex(v)
            
        for v in self.vertices:
            for u in self.adjacent(v):
                GT.add_edge(u,v)
        
        return GT
        
    def acyclic(self):
        found_BE = False
        
        for u in self.vertices:
            for v in self.adjacent(u):
                b1 = v.get('d') <= u.get('d')
                b2 = u.get('d') < u.get('f')
                b3 = u.get('f') <= v.get('f')
                if b1 and b2 and b3: 
                    found_BE = True
                    
        return not found_BE
        
    def topological_sort(self):
        self.DFS()
        if self.acyclic():
            result = sorted(self.vertices,key=lambda v: v.get('f'),reverse=True)
            return list(map(lambda v: v.get('id'),result))
        else:
            return None
    

    def checkPath(self, s, verticesTaken, current_vertex):
            #I'm going to leave the print statements from debugging in so that yall can see the 
            #process this takes. 
            print(s)
            print(current_vertex)
            print(verticesTaken)
            print("id To add to verticesTaken:", current_vertex.get('id'))
            #I believe the NoneType error was a result of doing:
                #list = list.append(x) instead of just list.append(x)
            #This fixed it.
            idToAddToVerticesTaken = current_vertex.get('id')
            verticesTaken.append(idToAddToVerticesTaken)
            print("Vertices Taken after append.", verticesTaken)
            
            #if length of sound to find is 0 return the path because it is consumed by one
                #each time a new acceptable edge is found
                #when it is 0 it returns the path of the full sound
            if len(s) == 0:
                return verticesTaken 
            
            #for each adjectent vertex, check if edge matches next element of sound (s[0])
            for v in self.adjacent(current_vertex):
                
                if self.edge_attributes[(current_vertex,v)]['sound'] == s[0]:
                    #if it's a match consume first element of sound to check for
                    s = s[1:len(s)]
                    #set the current vertex to the one taken by that edge
                    current_vertex = v 
                    
                    #recrusive call, s is consumed by one
                        #verticesTaken was updated prior to s len check
                        #current_vertex updated directly above comment
                    return self.checkPath(s, verticesTaken, current_vertex)
            #will return either the path or type None
                #the latter occuring when no path exists to satisfy sound to search for s.
                #maybe go back and check that this is indeed the case to justify the helper,
                    #like there is a possibility of an infinite loop, but I don't beleive that to be the case if no match

    def checkPathHelper(self, s, verticesTaken, current_vertex):
        #thing to Return is id's of the vertex's taken on path satisfying sound s. 
        thingToReturn = self.checkPath(s, verticesTaken, current_vertex)
        if thingToReturn == None:
            return 'NO-SUCH-PATH'
        else: return thingToReturn

In [6]:
def create_graph_4():
    G = DirectedGraph()
    
    for i in ['0','1','2','3','4']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2,sigma) in [('0','1','a'),('1','3','b'),('3','4','a'),('0','2','a')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2],sigma)
         
    return G
 

In [7]:
G4 = create_graph_4()
G4.checkPathHelper('aba', [], G4.id_to_v['0'])

aba
{'id': '0'}
[]
id To add to verticesTaken: 0
Vertices Taken after append. ['0']
ba
{'id': '1'}
['0']
id To add to verticesTaken: 1
Vertices Taken after append. ['0', '1']
a
{'id': '3'}
['0', '1']
id To add to verticesTaken: 3
Vertices Taken after append. ['0', '1', '3']

{'id': '4'}
['0', '1', '3']
id To add to verticesTaken: 4
Vertices Taken after append. ['0', '1', '3', '4']


['0', '1', '3', '4']